# Setup

In [56]:
# TODO: Verify that Python 3+ exists

import requests

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

# Setup pipeline paths

In [18]:
# Directory of the raw data files
_data_root = './data/'

# Create the dataset directory
!mkdir -p {_data_root}

PATH_RAW_DATA_JAN_2022 = f'{_data_root}NYC-Taxi-Dataset-2022-01.parquet'
PATH_RAW_DATA_FEB_2022 = f'{_data_root}NYC-Taxi-Dataset-2022-02.parquet'

URL_RAW_DATA_JAN_2022 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet'
URL_RAW_DATA_FEB_2022 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet'


# Load and prepare the dataset

In [10]:
def download_file(url: str, save_path: str) -> None:
    """Download any type of file and save it locally.

    Args:
        url (str): URL of data file
        save_path (str): Local path to save the file
    """

    response = requests.get(url)
    with open(save_path, 'wb') as file:
        file.write(response.content)

## Download data

In [22]:
try:
    download_file(URL_RAW_DATA_JAN_2022, PATH_RAW_DATA_JAN_2022)
    download_file(URL_RAW_DATA_FEB_2022, PATH_RAW_DATA_FEB_2022)
    print(f"Data downloaded, check the folder {_data_root}")
except Exception as e:
    print(f"Error: {e}")


Data downloaded, check the folder ./data/


## Load data

In [45]:
df_january = pd.read_parquet(PATH_RAW_DATA_JAN_2022)
df_january

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.50,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.00,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.50,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.00,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.50,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463926,2,2022-01-31 23:36:53,2022-01-31 23:42:51,NaN,1.32,NaN,None,90,170,0,8.00,0.0,0.5,2.39,0.0,0.3,13.69,NaN,NaN
2463927,2,2022-01-31 23:44:22,2022-01-31 23:55:01,NaN,4.19,NaN,None,107,75,0,16.80,0.0,0.5,4.35,0.0,0.3,24.45,NaN,NaN
2463928,2,2022-01-31 23:39:00,2022-01-31 23:50:00,NaN,2.10,NaN,None,113,246,0,11.22,0.0,0.5,2.00,0.0,0.3,16.52,NaN,NaN
2463929,2,2022-01-31 23:36:42,2022-01-31 23:48:45,NaN,2.92,NaN,None,148,164,0,12.40,0.0,0.5,0.00,0.0,0.3,15.70,NaN,NaN


In [39]:
df_january.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

### Question 1. 
Read the data for January. 

**How many columns are there?**

In [46]:
print(f"The number of columns is: {len(df_january.columns)}")

The number of columns is: 19


In [50]:

df_january['duration'] = df_january.tpep_dropoff_datetime - df_january.tpep_pickup_datetime
df_january.duration = df_january.duration.apply(lambda td: td.total_seconds() / 60)

### Question 2. Computing duration
Compute the duration variable. It should contain the duration of a ride in minutes.

**What's the standard deviation of the trips duration in January?**

In [48]:
df_january['duration']

0          17.816667
1           8.400000
2           8.966667
3          10.033333
4          37.533333
             ...    
2463926     5.966667
2463927    10.650000
2463928    11.000000
2463929    12.050000
2463930    27.000000
Name: duration, Length: 2463931, dtype: float64

In [51]:
std_deviation = df_january['duration'].std()

print(f"Standard Deviation for column `duration` is: {std_deviation}")

Standard Deviation for column `duration` is: 46.44530513776802


### Question 3. Dropping outliers
Check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

**What fraction of the records left after you dropped the outliers?**

In [54]:
before_drop = len(df_january)
print(f'Records before drop: {before_drop}')
df_january = df_january[(df_january.duration >= 1) & (df_january.duration <= 60)]

after_drop = len(df_january)
print(f'Records after dropping: {after_drop}')

print(f'Fraction of the records left after dropped the outliers: {100 - (after_drop/before_drop)}')

Records before drop: 2421440
Records after dropping: 2421440
Fraction of the records left after dropped the outliers: 99.0


### Question 4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer
* Get a feature matrix from it

**What's the dimensionality of this matrix (number of columns)?**

In [57]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_january[categorical] = df_january[categorical].astype(str)

train_dicts = df_january[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df_january[target].values


In [59]:

# Convert the feature matrix to an array
feature_array = X_train.toarray()

# Get the feature names
feature_names = dv.get_feature_names_out()

# Print the feature matrix and feature names
print("Feature Matrix:")
print(X_train)
print("\nFeature Array:")
print(feature_array)
print("\nFeature Names:")
print(feature_names)


Feature Matrix:
  (0, 148)	1.0
  (0, 306)	1.0
  (0, 515)	3.8
  (1, 198)	1.0
  (1, 406)	1.0
  (1, 515)	2.1
  (2, 71)	1.0
  (2, 332)	1.0
  (2, 515)	0.97
  (3, 226)	1.0
  (3, 275)	1.0
  (3, 515)	1.09
  (4, 68)	1.0
  (4, 482)	1.0
  (4, 515)	4.3
  (5, 66)	1.0
  (5, 301)	1.0
  (5, 515)	10.3
  (6, 247)	1.0
  (6, 403)	1.0
  (6, 515)	5.07
  (7, 56)	1.0
  (7, 408)	1.0
  (7, 515)	2.02
  (8, 148)	1.0
  :	:
  (2421431, 515)	2.42
  (2421432, 16)	1.0
  (2421432, 417)	1.0
  (2421432, 515)	7.51
  (2421433, 234)	1.0
  (2421433, 345)	1.0
  (2421433, 515)	8.5
  (2421434, 251)	1.0
  (2421434, 393)	1.0
  (2421434, 515)	1.63
  (2421435, 76)	1.0
  (2421435, 506)	1.0
  (2421435, 515)	1.32
  (2421436, 234)	1.0
  (2421436, 268)	1.0
  (2421436, 515)	4.19
  (2421437, 159)	1.0
  (2421437, 274)	1.0
  (2421437, 515)	2.1
  (2421438, 69)	1.0
  (2421438, 312)	1.0
  (2421438, 515)	2.92
  (2421439, 88)	1.0
  (2421439, 352)	1.0
  (2421439, 515)	8.94

Feature Array:
[[0.   0.   0.   ... 0.   0.   3.8 ]
 [0.   0.   0.   ... 

In [62]:
print("Matrix number of columns:")
print(X_train.shape[1]-1)

Matrix number of columns:
515


In [ ]:

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)